## Implementación de sistema SIMETRIK en base de datos CLAP Y BANSUR CON SQLite3
### Torres Miguel Alejandro
### Fecha: 09/07/2023
   
#### Se realizan las importaciones necesarias y se cargan los archivos al entorno.

In [15]:
import csv
import sqlite3


def createTable(name, header, rows, cursor):
    newHeader = []
    for col in header:
        clean_col = col.replace("\ufeff", '')  # Eliminar caracteres especiales
        newHeader.append(clean_col)
    # Crear la tabla con el nombre especificado        
    create_table_query = 'CREATE TABLE {} ({})'.format(name, ', '.join(['"{}"'.format(col) for col in newHeader]))
    cursor.execute(create_table_query)

    # Insertar los datos en la tabla
    insert_data_query = 'INSERT INTO {} VALUES ({})'.format(name, ', '.join(['?' for _ in newHeader]))
    cursor.executemany(insert_data_query, rows)

def modifyTable(name, nameR, header, rows, cursor):
    newHeader = []
    for col in header:
        clean_col = col.replace("\ufeff", '')  # Eliminar caracteres especiales
        newHeader.append(clean_col)

    # Crear la tabla con el nombre especificado
    create_table_query = 'CREATE TABLE {} (ID INTEGER PRIMARY KEY AUTOINCREMENT DEFAULT 0, {}, CRUZADO DEFAULT "-")'.format(name, ', '.join(['"{}"'.format(col) for col in newHeader]))
    cursor.execute(create_table_query)

    # Insertar los datos en la tabla
    insert_data_query = 'INSERT INTO {} ({},CRUZADO) VALUES ({},"{}")'.format(name, ', '.join(header), ', '.join(['?' for _ in newHeader]),'-')
    cursor.executemany(insert_data_query, rows)


    # Borrar la tabla nameR si existe
    drop_table_query = 'DROP TABLE IF EXISTS {}'.format(nameR)
    cursor.execute(drop_table_query)


    # Renombrar la tabla name a nameR
    rename_table_query = 'ALTER TABLE {} RENAME TO {}'.format(name, nameR)
    cursor.execute(rename_table_query)


#### Se crean las tablas SQL para gestionar las bases de datos

In [16]:
clap_file_path = '/home/aletorres/OneDrive/AleOD_/WORK/Prueba Técnica-selected/CLAP.csv'
bansur_file_path = '/home/aletorres/OneDrive/AleOD_/WORK/Prueba Técnica-selected/BANSUR.csv'

with open(clap_file_path, 'r') as clapfile:
    clap_data = csv.reader(clapfile)
    clap_header = next(clap_data)  # Lee la primera línea (encabezados)
    clap_rows = list(clap_data)  # Lee las filas restantes

with open(bansur_file_path, 'r') as bansurfile:
    bansur_data = csv.reader(bansurfile)
    bansur_header = next(bansur_data)  # Lee la primera línea (encabezados)
    bansur_rows = list(bansur_data)  # Lee las filas restantes


conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

createTable('clap', clap_header, clap_rows, cursor)
createTable('bansur', bansur_header, bansur_rows, cursor)


#### Primer consulta a las bases de datos

In [17]:
query = '''
            SELECT *
            FROM clap
        '''
cursor.execute(query)

for row in (cursor.fetchmany(10)):
    print(row)
print('\n')

query = '''SELECT * FROM bansur '''
cursor.execute(query)

for row in (cursor.fetchmany(10)):
    print(row)

('443715', '0190', 'CANCELADA', '21944', '2020-11-01 07:50:25.000', '136719', '047100185686', '2020-11-05')
('802414', '6615', 'PAGADA', '42.51', '2020-11-01 04:35:04.000', '053042', '029607730901', '2020-11-02')
('818156', '0672', 'NO APLICA', '20.8', '2020-11-01 04:35:04.000', '', '497363207741', '2020-11-02')
('954015', '5581', 'PAGADA', '41', '2020-11-01 04:35:06.000', '013616', '441622557487', '2020-11-02')
('954015', '2538', 'PAGADA', '68', '2020-11-01 04:35:06.000', '013602', '266554946896', '2020-11-02')
('954015', '2581', 'PAGADA', '144', '2020-11-01 04:35:06.000', '013626', '106967585923', '2020-11-02')
('818156', '8094', 'PAGADA', '189.5', '2020-11-01 04:35:06.000', '053064', '890554602678', '2020-11-02')
('946891', '1670', 'NO APLICA', '776.6', '2020-11-01 04:35:06.000', 'H37254', '249718452281', '2020-11-02')
('958758', '4254', 'PAGADA', '27.54', '2020-11-01 04:35:08.000', '151121', '243262174009', '2020-11-02')
('930288', '8524', 'PAGADA', '88', '2020-11-01 04:35:08.000',

#### 1. Escriba el código de SQL que le permite conocer el monto y la cantidad de las transacciones que SIMETRIK considera como conciliables para la base de CLAP

In [18]:
query = '''SELECT SUM(MONTO), COUNT(*) FROM clap WHERE TIPO_TRX = 'PAGADA' ORDER BY FECHA_TRANSACCION'''

cursor.execute(query)
result_clap = cursor.fetchone()

print("Monto : $",round(result_clap[0],2))
print("Cantidad:",result_clap[1])

Monto : $ 61050819.41
Cantidad: $ 147331


#### 2. Escriba el código de SQL que le permite conocer el monto y la cantidad de las transacciones que SIMETRIK considera como conciliables para la base de BANSUR

In [40]:
query = '''SELECT SUM(MONTO), COUNT(*) FROM bansur WHERE TIPO_TRX = 'PAGO' ORDER BY FECHA_TRANSACCION'''

cursor.execute(query)
result_bansur = cursor.fetchone()

print("Monto :$",round(result_bansur[0],2))
print("Cantidad:",result_bansur[1])

Monto :$ 54053911.94
Cantidad: 132338


#### 3. ¿Cómo se comparan las cifras de los puntos anteriores respecto de las cifras totales en las fuentes desde un punto de vista del negocio?

In [20]:
query = '''SELECT SUM(MONTO), COUNT(*) FROM bansur ORDER BY FECHA_TRANSACCION'''

cursor.execute(query)
result_bansur_all = cursor.fetchone()

print("Monto total registrado en BANSUR: $", round(result_bansur_all[0],2))
print("Cantidad total de trasacciones registradas en BANSUR:",result_bansur_all[1])

query = '''SELECT SUM(MONTO), COUNT(*) FROM clap ORDER BY FECHA_TRANSACCION'''

cursor.execute(query)
result_clap_all = cursor.fetchone()
print("\nMonto total registrado en CLAP: $", round(result_clap_all[0],2))
print("Cantidad total de trasacciones registradas en CLAP",result_clap_all[1])

Monto total registrado en BANSUR: $ 53977030.03
Cantidad total de trasacciones registradas en BANSUR: $ 132396

Monto total registrado en CLAP: $ 73736800.92
Cantidad total de trasacciones registradas en CLAP 163549


#### Para comparar los resultados totales con las busquedas filtradas, realizamos un porcentaje de los valores totales sobre los obtenidos.

In [21]:
print("Porcentaje de transacciones pagadas sobre las totales en CLAP : %",round(result_clap[0]/result_clap_all[0],3))
print("Porcentaje de las cantidades pagadas sobre las cantidades totales en CLAP: %",round(result_clap[1]/result_clap_all[1],3))

print("\nPorcentaje de transacciones pagadas sobre las totales en BANSUR : %",round(result_bansur[0]/result_bansur_all[0],3))
print("Porcentaje de las cantidades pagadas sobre las cantidades totales en BANSUR  : %",round(result_bansur[1]/result_bansur_all[1],3))

Porcentaje de transacciones pagadas sobre las totales en CLAP : % 0.828
Porcentaje de las cantidades pagadas sobre las cantidades totales en CLAP: % 0.901

Porcentaje de transacciones pagadas sobre las totales en BANSUR : % 1.001
Porcentaje de las cantidades pagadas sobre las cantidades totales en BANSUR  : % 1.0


#### 4. Teniendo en cuenta los criterios de cruce entre ambas bases conciliables, escriba una sentencia de SQL que contenga la información de CLAP y BANSUR: Agregue una columna en la que se evidencie si la transacción cruzó o no con su contrapartida y una columna en la que se inserte un ID autoincremental para el control de la conciliación.
  
Bases conciliables:

In [22]:
query = '''SELECT * FROM clap WHERE TIPO_TRX = 'PAGADA' ORDER BY FECHA_TRANSACCION'''
cursor.execute(query)
clap_conciliable= cursor.fetchmany(10)

for row in clap_conciliable:
    print(row)
    
print('\n')

query = '''SELECT * FROM bansur WHERE TIPO_TRX = 'PAGO' ORDER BY FECHA_TRANSACCION'''
cursor.execute(query)
clap_conciliable= cursor.fetchmany(10)

for row in clap_conciliable:
    print(row)

('954015', '0957', 'PAGADA', '52.5', '2020-11-01 00:00:00.000', '150741', '345794185336', '2020-11-01')
('497797', '5698', 'PAGADA', '150', '2020-11-01 00:00:00.000', '392762', '123262279716', '2020-11-01')
('404826', '5177', 'PAGADA', '171', '2020-11-01 00:00:00.000', '018295', '167018133724', '2020-11-01')
('403958', '2775', 'PAGADA', '75', '2020-11-01 00:00:00.000', '017647', '741123866254', '2020-11-01')
('497797', '2152', 'PAGADA', '126', '2020-11-01 00:00:00.000', '396307', '137520451938', '2020-11-01')
('975367', '6725', 'PAGADA', '325', '2020-11-01 00:00:00.000', '012017', '722806390634', '2020-11-01')
('971546', '0569', 'PAGADA', '50', '2020-11-01 00:00:02.000', '005248', '370020923294', '2020-11-01')
('930288', '7755', 'PAGADA', '155', '2020-11-01 00:00:02.000', '022015', '679067777866', '2020-11-01')
('442753', '2739', 'PAGADA', '40', '2020-11-01 00:00:02.000', '000412', '197845506377', '2020-11-01')
('954015', '6214', 'PAGADA', '70', '2020-11-01 00:00:02.000', '151479', '62

In [23]:

modifyTable('clapTemp', 'clap', clap_header, clap_rows, cursor)
modifyTable('bansurTemp','bansur', bansur_header, bansur_rows, cursor)

query = '''
    UPDATE clap
    SET FECHA_TRANSACCION = strftime('%Y%m%d', FECHA_TRANSACCION)
'''

cursor.execute(query)

query = '''
    UPDATE clap
    SET CRUZADO = 'si'
    FROM bansur
    WHERE clap.ID_BANCO = bansur.ID_ADQUIRIENTE
    AND clap.TIPO_TRX = 'PAGADA'
    AND bansur.TIPO_TRX = 'PAGO'
    AND clap.INICIO6_TARJETA || FINAL4_TARJETA = bansur.TARJETA
    AND (clap.MONTO = bansur.MONTO OR ABS(clap.MONTO - bansur.MONTO) <= 0.99)
    AND clap.FECHA_TRANSACCION = bansur.FECHA_TRANSACCION
'''

cursor.execute(query)

query = '''
    SELECT *
    FROM clap
    WHERE CRUZADO = 'si'
    ORDER BY FECHA_TRANSACCION
'''

cursor.execute(query)
clap_cruzado = cursor.fetchmany(5)  # Obtener las primeras 10 líneas

for row in clap_cruzado:
    print(row)
    
query = '''
    UPDATE bansur
    SET CRUZADO = 'si'
    FROM clap
    WHERE clap.ID_BANCO = bansur.ID_ADQUIRIENTE
    AND clap.TIPO_TRX = 'PAGADA'
    AND bansur.TIPO_TRX = 'PAGO'
    AND clap.INICIO6_TARJETA || FINAL4_TARJETA = bansur.TARJETA
    AND (clap.MONTO = bansur.MONTO OR ABS(clap.MONTO - bansur.MONTO) <= 0.99)
    AND clap.FECHA_TRANSACCION = bansur.FECHA_TRANSACCION
'''

cursor.execute(query)

query = '''
    SELECT *
    FROM bansur
    WHERE CRUZADO = 'si'
    ORDER BY FECHA_TRANSACCION
'''

cursor.execute(query)
bansur_cruzado = cursor.fetchmany(5)  # Obtener las primeras 10 líneas

print("\n")
for row in clap_cruzado:
    print(row)


(6281, '954015', '0108', 'PAGADA', '500', '20201101', '193186', '041640116604', '2020-11-02', 'si')
(6282, '954015', '6530', 'PAGADA', '130', '20201101', '193429', '041467739202', '2020-11-02', 'si')
(6283, '497797', '2844', 'PAGADA', '2100', '20201101', '127882', '074878446381', '2020-11-02', 'si')
(6284, '958758', '1814', 'PAGADA', '119', '20201101', '183237', '042848711911', '2020-11-02', 'si')
(6285, '404826', '6379', 'PAGADA', '450', '20201101', '024461', '890542723340', '2020-11-02', 'si')


(6281, '954015', '0108', 'PAGADA', '500', '20201101', '193186', '041640116604', '2020-11-02', 'si')
(6282, '954015', '6530', 'PAGADA', '130', '20201101', '193429', '041467739202', '2020-11-02', 'si')
(6283, '497797', '2844', 'PAGADA', '2100', '20201101', '127882', '074878446381', '2020-11-02', 'si')
(6284, '958758', '1814', 'PAGADA', '119', '20201101', '183237', '042848711911', '2020-11-02', 'si')
(6285, '404826', '6379', 'PAGADA', '450', '20201101', '024461', '890542723340', '2020-11-02', 's

#### 5. Diseñe un código que calcule el porcentaje de transacciones de la base conciliable de CLAP cruzó contra la liquidación de BANSUR.


In [46]:
query = '''SELECT COUNT(*) FROM clap WHERE CRUZADO = 'si' '''

cursor.execute(query)
count_conciliable_clap = cursor.fetchall()
print("Cantidad de transacciones concilliables en CLAP cruzadas:",count_conciliable_clap[0][0])

print("\nMonto total registrado en BANSUR:",result_bansur_all[1])
print("\nPorcentaje de transacciones de CLAP conciliables cruzadas en la liquidación de BANSUR:%",
         round((count_conciliable_clap[0][0]/result_bansur_all[1]),2))

Cantidad de transacciones concilliables en CLAP cruzadas: 96355

Monto total registrado en BANSUR: 132396

Porcentaje de transacciones de CLAP conciliables cruzadas en la liquidación de BANSUR:% 0.73


#### 6. Diseñe un código que calcule el porcentaje de transacciones de la base conciliable de BANSUR no cruzó contra la liquidación de CLAP.

In [48]:
query = '''SELECT COUNT(*) FROM bansur WHERE CRUZADO = 'si' '''

cursor.execute(query)
count_conciliable_bansur = cursor.fetchall()
print("Cantidad de transacciones concilliables en CLAP cruzadas:",count_conciliable_bansur[0][0])

print("\nMonto total registrado en BANSUR:",result_clap_all[1])
print("\nPorcentaje de transacciones de BANSUR conciliables cruzadas en la liquidación de CLAP:%",
         round((count_conciliable_bansur[0][0]/result_clap_all[1]),2))

Cantidad de transacciones concilliables en CLAP cruzadas: 96355

Monto total registrado en BANSUR: 163549

Porcentaje de transacciones de BANSUR conciliables cruzadas en la liquidación de CLAP:% 0.59
